# SSWEI Calculation Workflow

This notebook demonstrates the SSWEI (Standardized Snow Water Equivalent Index) calculation workflow for the Snow Drought Index package. It covers loading data, preparing seasonal data, calculating the SSWEI, and classifying drought conditions.

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy.integrate import trapz
from scipy.stats import norm

# Import snowdroughtindex package
from snowdroughtindex.core import data_preparation, gap_filling, sswei
from snowdroughtindex.utils import visualization, io

## 1. Data Loading

First, we'll load the gap-filled SWE data that will be used for SSWEI calculation.

In [ ]:
# Define data paths
swe_path = '../data/processed/swe_gapfilled.nc'

# Load data using the implemented functions
swe_data = data_preparation.load_swe_data(swe_path)

# Convert to DataFrame for processing
swe_df = data_preparation.preprocess_swe(swe_data)

# Display the first few rows of the data
swe_df.head()

## 2. Calculate Basin-Wide Mean SWE

Next, we'll calculate the daily mean SWE across all stations in the basin.

In [ ]:
# Ensure the DataFrame has a time index
if 'time' in swe_df.columns:
    swe_df = swe_df.set_index('time')

# Identify SWE columns (excluding metadata columns)
swe_columns = [col for col in swe_df.columns if col not in ['station_id', 'lat', 'lon', 'elevation']]

# Calculate daily mean SWE across all stations
daily_mean = pd.DataFrame({
    'date': swe_df.index,
    'mean_SWE': swe_df[swe_columns].mean(axis=1)
})

# Display the first few rows of the daily mean data
daily_mean.head()

## 3. Visualize Daily Mean SWE

Let's visualize the daily mean SWE values to understand the seasonal patterns.

In [ ]:
# Plot the daily mean SWE values
plt.figure(figsize=(12, 6))
plt.plot(daily_mean['date'], daily_mean['mean_SWE'])
plt.xlabel('Date')
plt.ylabel('Mean SWE (mm)')
plt.title('Daily Mean SWE Values')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Prepare Seasonal Data

Now we'll prepare the seasonal data by filtering for complete snow seasons.

In [ ]:
# Define season parameters
start_month = 11  # November
start_day = 1
end_month = 4    # April
end_day = 30
min_swe = 15     # Minimum SWE to consider as the start of the snow season

# Prepare seasonal data
season_data = sswei.prepare_season_data(
    daily_mean, 
    start_month=start_month, 
    start_day=start_day, 
    end_month=end_month, 
    end_day=end_day, 
    min_swe=min_swe
)

# Display the first few rows of the seasonal data
season_data.head()

## 5. Calculate Seasonal Integration

Next, we'll calculate the seasonal integration of SWE values.

In [ ]:
# Calculate seasonal integration
integrated_data_season = sswei.calculate_seasonal_integration(season_data, start_month=start_month)

# Display the integrated data
integrated_data_season

## 6. Calculate SSWEI

Now we'll calculate the SSWEI values and classify drought conditions.

In [ ]:
# Calculate SSWEI directly from daily mean data
sswei_results = sswei.calculate_sswei(
    daily_mean, 
    start_month=start_month, 
    start_day=start_day, 
    end_month=end_month, 
    end_day=end_day, 
    min_swe=min_swe
)

# Display the SSWEI results
sswei_results[['season_year', 'total_SWE_integration', 'Gringorten_probabilities', 'SWEI', 'Drought_Classification']]

## 7. Visualize SSWEI Results

Let's visualize the SSWEI values and drought classifications.

In [ ]:
# Plot SSWEI values with drought classification thresholds
sswei_plot = sswei.plot_sswei(sswei_results)
plt.show()

## 8. Analyze Drought Conditions

Let's analyze the drought conditions over the years.

In [ ]:
# Count occurrences of each drought classification
drought_counts = sswei_results['Drought_Classification'].value_counts()

# Plot the counts
plt.figure(figsize=(10, 6))
drought_counts.plot(kind='bar', color='skyblue')
plt.title('Frequency of Drought Classifications')
plt.xlabel('Drought Classification')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate percentage of each classification
drought_percentage = (drought_counts / len(sswei_results) * 100).round(1)
print("Percentage of each drought classification:")
for classification, percentage in drought_percentage.items():
    print(f"{classification}: {percentage}%")

## 9. Identify Drought Years

Let's identify the years with drought conditions.

In [ ]:
# Filter for drought years (SWEI < -0.5)
drought_years = sswei_results[sswei_results['SWEI'] < -0.5]

# Sort by SWEI to see the most severe droughts first
drought_years = drought_years.sort_values('SWEI')

# Display drought years
print("Years with drought conditions (SWEI < -0.5):")
drought_years[['season_year', 'SWEI', 'Drought_Classification']]

## 10. Save SSWEI Results

Finally, let's save the SSWEI results for future analysis.

In [ ]:
# Save SSWEI results to CSV
sswei_results.to_csv('../data/processed/sswei_results.csv', index=False)

print("SSWEI results saved to '../data/processed/sswei_results.csv'")

## 11. Summary

In this notebook, we've demonstrated the SSWEI calculation workflow for the Snow Drought Index package. We've loaded gap-filled SWE data, calculated daily mean SWE values, prepared seasonal data, calculated the SSWEI, classified drought conditions, and analyzed the results.

The workflow uses the following key functions from the `sswei` module:
- `prepare_season_data()` for filtering complete snow seasons
- `calculate_seasonal_integration()` for integrating SWE values over the season
- `calculate_sswei()` for calculating SSWEI values and classifying drought conditions
- `plot_sswei()` for visualizing SSWEI values with drought classification thresholds

These functions provide a standardized and reusable way to calculate the SSWEI and classify drought conditions based on SWE data.